In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [2]:
# データのパス
data_path = '/Users/manqueenmannequin/TW_otameshi/project_root/data/processed/android_cleaned_mecab.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

データサイズ: (1000, 18)


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review,positive_review_keywords,negative_review_keywords
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し..."
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']"
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[]
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']"
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[]


In [3]:
# データのnegative_review_keywordsに対してだけ，Topic Modelingを行う

negative_review_keywords = df['negative_review_keywords'].dropna().tolist()

In [4]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [5]:
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# umapとhdbscanのモデルを作成
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)

# BERTopicモデルを作成

topic_model = BERTopic(
    language="japanese",
    calculate_probabilities=False,
    verbose=True,
    nr_topics="20",
    umap_model=umap_model, 
    hdbscan_model=hdbscan_model
    )

topics, probs = topic_model.fit_transform(negative_review_keywords)

2024-12-17 00:26:49,827 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-12-17 00:26:56,807 - BERTopic - Embedding - Completed ✓
2024-12-17 00:26:56,807 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2024-12-17 00:27:01,303 - BERTopic - Dimensionality - Completed ✓
2024-12-17 00:27:01,304 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-17 00:27:01,324 - BERTopic - Cluster - Completed ✓
2024-12-17 00:27:01,324 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-17 00:27:01,338 - BERTopic - Representation - Completed ✓
2024-12-17 00:27:01,338 - BERTopic - Topic reduction - Reducing number of topics
2024-12-17 00:27:01,353 - BERTopic - Topic reduction - Reduced number of topics from 13 to 13


In [7]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,137,-1_いる_こと_なっ_ポイント,"[いる, こと, なっ, ポイント, 不満, ため, ゲーム, 感じ, よう, アプリ]","[['ポイント', '消え', '交換', 'でき', 'こと', '詐欺', 'アプリ',..."
1,0,369,0____,"[, , , , , , , , , ]","[[], [], []]"
2,1,98,1_広告_多く_する_表示,"[広告, 多く, する, 表示, れる, 閉じる, ボタン, こと, 詐欺, ロード]","[['広告', '多', 'すぎる'], ['広告', '多い'], ['2', '重', ..."
3,2,86,2_動画_再生_視聴_ポイント,"[動画, 再生, 視聴, ポイント, でき, 時間, こと, ロード, ため, 問題]","[['動画', '再生', 'さ', 'れ'], ['動画', '再生', 'さ', 'れ'..."
4,3,67,3_ポイント_交換_不満_こと,"[ポイント, 交換, 不満, こと, 貯まら, でき, 貯め, にくい, 減る, 貯まり]","[['ポイント', '貯まら', 'こと', '不満', '感じ', 'いる'], ['ポイ..."
5,4,59,4_fi_wi_wifi_接続,"[fi, wi, wifi, 接続, 繋がら, タウン, いる, スポット, でき, ある]","[['タウン', 'Wi-Fi', 'Wi-Fi', '使え', 'こと', 'ない'], ..."
6,5,40,5_クイズ_難しい_難し_難点,"[クイズ, 難しい, 難し, 難点, フリーズ, 上手く, わから, 感じる, ない, 使い]","[['クイズ', '難しい', '点', '挙げ', 'られ', 'い'], ['クイズ',..."
7,6,33,6_接続_自動_繋がり_悪い,"[接続, 自動, 繋がり, 悪い, vpn, つながり, でき, スタンプ, wifi, 問題]","[['接続', '自動', 'でき', 'こと', 'ある'], ['自動', '接続', ..."
8,7,32,7_アプリ_再起動_表示_する,"[アプリ, 再起動, 表示, する, こと, アップデート, 停止, 古い, 改善, 問題]","[['アプリ', '問題', 'あり'], ['アプリ', 'アップデート', '必要', ..."
9,8,21,8_問題_位置情報_ある_許可,"[問題, 位置情報, ある, 許可, いる, 改悪, れる, 設定, スタンプ, 頻繁]","[['位置', '確認', '許可', 'し', 'いる', '何', '十', 'メートル..."


In [8]:
topic_model.visualize_topics()

In [9]:
topic_model.visualize_hierarchy()

In [6]:
topic_model.visualize_barchart(
    top_n_topics=14,
    width=500,  # 幅を広げる
)